In [54]:
import load
from tree import tmtree
import numpy as np
import utils.sampling as smp
import tree.halomodule as hmo 


from astropy.cosmology import WMAP7 as cosmo

wdir = '/home/hoseung/Work/data/05427/'
nout = 186
for nout in range(1,10):
    info = load.info.Info(nout=nout, base=wdir, load=True)    
    print(cosmo.lookback_time(info.zred).value)
    
    #print(info.tGyr)
#hh = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, load=True)

13.681712286550637
13.613061656496194
13.556791131055624
13.478121881730736
13.416971734869152
13.329089208282788
13.194865848327336
13.152362296253386
13.108839406157436


In [27]:
import matplotlib.pyplot as plt
plt.hist(hh.data['x'])
plt.show()


In [44]:
def mk_gal(region, out_q, info, i, final_gal, galid = None,
            save=False, rscale=0.3, verbose=False, galaxy_plot_dir='./',
            rscale_lambda=2.0, npix_lambda=50, npix=400, galaxy_plot=False,
            method_com=2, follow_most_bound=None):
    """
    Direct plot,
    Create galaxy, 
    Calculate lambda_r (using Cappellari 2003)
    Draw ed map of galaxy.
    
    
    NOTE
        x,y,z, radius  = region
        + id from halodata. 
    
    """
    from galaxymodule import galaxy
    import utils.sampling as smp

#    print("IDs:", id(star), id(dm), id(cell))

    gal_out = {"id":0, "xc":0.0, "yc":0.0, "zc":0.0,
               "vx":0.0, "vy":0.0, "vz":0.0,
               "mstar":0.0, "nstar":0.0, "mgas":0.0,
               "lambda_arr":[], "lambda_r":0, "rgal":0, "final":final_gal,
               "rhalo":region['radius'], "boxtokpc":info.pboxsize*1000}

    #                "rhalo":halodata['rvir'], "boxtokpc":info.pboxsize*1000}
    # Direct plot ---------------------------------------------------------
    #region = smp.set_region(xc=halodata['x'],
    #                        yc=halodata['y'],
    #                        zc=halodata['z'],
    #                        radius = halodata['rvir'])

    if galid == None:
        galid = 1

    if galaxy_plot:
        if follow_most_bound is None:
            plot_region(star, region, galid, s.info.pboxsize, npix=300, ids=[], color=None)
        else:
            import utils.match as mtc
            f = plt.figure()
            ax = f.add_subplot(111)
            ax.scatter(star['x'], star['y'])
            ind = mtc.match_list_ind(star['id'], follow_most_bound)
            
            ax.scatter(star['x'][ind], star['y'][ind], c='red')
            plt.show()

        

    #Create galaxy ---------------------------------------------------------
    gal = galaxy.Galaxy(halodata, radius_method='eff', info=info)
    is_gal = gal.mk_gal(star=star, dm=dm, cell=cell,
               rscale=rscale, verbose=verbose, method_com=method_com,
               follow_bp=follow_most_bound)
    #-----------------------------------------------------------------------    
    print(gal.id, "IS_GAL",is_gal)
    if not is_gal:
        print(gal.id, " Not a good galaxy")
        out_q.put(gal_out)
    else:
        # Save to catalog -------------------------------------------------------
#        print("Good galaxy, R eff:", gal.reff)
        gal.cal_lambda_r(npix=npix_lambda, method=1, rscale=rscale_lambda) # calculate within 1.0 * reff    
        # Calculate lambda_r ---------------------------------------------------

        gal.plot_gal(fn_save = galaxy_plot_dir + str(nout).zfill(3) \
                             + "_" + str(final_gal).zfill(5) + "_"  \
                             + str(gal.id) + ".png", ioff=True)
#       gal.save_gal(base=wdir)

        # save in a dict.
        gal_out['mstar'] = gal.mstar
        gal_out['mgas'] = gal.mgas
        gal_out['nstar'] = gal.nstar
        gal_out['id'] = gal.id
        gal_out['xc'] = gal.xc * info.pboxsize
        gal_out['yc'] = gal.yc * info.pboxsize
        gal_out['zc'] = gal.zc * info.pboxsize
        gal_out['vx'] = gal.vxc * info.kms
        gal_out['vy'] = gal.vyc * info.kms
        gal_out['vz'] = gal.vzc * info.kms        
        gal_out['lambda_arr'] = gal.lambda_arr
        gal_out['lambda_r'] = gal.lambda_r
        gal_out['rgal'] = gal.reff# * info.pboxsize * 1000.0 # in kpc
        out_q.put(gal_out)
    print("mk_gal done")
    return gal

5.92901803531337e-10

In [32]:
def find_halo(allhalo, xc, yc, zc, search_radius, mvir):
    """
        Returns index of the most probable progenitor.
    """

    ind_near = np.where( (allhalo.data['x'] - allhalo.data['rvir'] < xc + search_radius) & 
                         (allhalo.data['x'] + allhalo.data['rvir'] > xc - search_radius) &
                         (allhalo.data['y'] - allhalo.data['rvir'] < yc + search_radius) & 
                         (allhalo.data['y'] + allhalo.data['rvir'] > yc - search_radius) &
                         (allhalo.data['z'] - allhalo.data['rvir'] < zc + search_radius) & 
                         (allhalo.data['z'] + allhalo.data['rvir'] > zc - search_radius) )

    print(ind_near)
    mvir_dist = np.abs(allhalo.data['mvir'][ind_near] - mvir)   
    print(mvir_dist)
    return ind_near[np.argsort(mvir_dist)[0]]


xc,yc,zc = 0.55, 0.35, 0.35
search_radius = 0.01
mvir = 1e13

h = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info)
h.derive_from(hh, hh.data['id'][find_halo(hh, xc, yc, zc, search_radius, mvir)])

(array([790]),)
[  9.74321854e+12]


#### Bound particle Vs TMtree

In [45]:
nout_fi = 187
nout_ini = 100
nout_ini0 = 100

r_cluster_scale = 2.0 # maximum radius inside which galaxies are searched for

## halo -----------------------------------------------------------
m_halo_min = 1e10 # minimum halo mass above which galaxies are searched for. 
dir_out = wdir + 'catalog/'

# nout_halo = 122 == nout 10, nout_halo = 0   == nout 132
tt = tmtree.load(work_dir=wdir, filename="halo/TMtree.fits")
tfin = tt[np.where(tt['NOUT'] == 0)]
tini = tt[np.where(tt['NOUT'] == nout_fi - nout_ini)]


hh = hmo.Halo(base=wdir, nout=nout_fi, halofinder='HM', info=info, load=True)
i_center = np.where(hh.data['np'] == max(hh.data['np']))
i_satellites = smp.extract_halos_within(hh.data, i_center, scale=r_cluster_scale)
print("Total {0} halos \n{1} halos are selected".format(
      len(i_satellites),sum(i_satellites)))

# halos found inside the cluster and has tree back to nout_ini
large_enugh = hh.data['mvir'] > m_halo_min
halo_list = hh.data['id'][i_satellites * large_enugh]
h_ind_ok, halo_ok = tmtree.check_tree_complete(tt, 0, nout_fi - nout_ini0, halo_list)
print(len(halo_ok), "halos left")
final_gal = halo_ok[:,0]
ngals = len(final_gal)


Total 1959 halos 
649 halos are selected
295 halos left


In [43]:
halo_ok[20]

array([736, 725, 726, 717, 707, 694, 708, 708, 706, 704, 722, 717, 726,
       728, 730, 741, 733, 724, 285, 291, 291, 289, 281, 286, 283, 287,
       280, 292, 288, 292, 282, 280, 288, 302, 295, 295, 300, 296, 730,
       723, 307, 304, 317, 314, 319, 321, 332, 346, 351, 344, 365, 363,
       363, 364, 375, 363, 380, 375, 371, 386, 729, 371, 383, 377, 381,
       380, 705, 696, 696, 698, 701, 689, 709, 697, 695, 691, 695, 681,
       686, 676, 678, 670, 680, 676, 663, 686, 682, 675])

In [ ]:
# load particle

# find most bound 

# 

